In [1]:
import pathlib
import os
import subprocess
import shutil
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [2]:
data_dir = pathlib.Path('D:\\paperback\\LABELED-BCs\\simple-cov\\')

In [13]:
out_dir = pathlib.Path('../data')
os.makedirs(out_dir, exist_ok=True)

In [4]:
def get_bc_program_paths(data_dir):
    for par_dir in data_dir.iterdir():
        for file in par_dir.iterdir():
            if str(file).endswith('.bc'):
                yield file

In [ ]:
for bc_path in tqdm(get_bc_program_paths(data_dir)):
    program_name = bc_path.name.split('-')[0]
    out_program_dir = out_dir / program_name
    os.makedirs(out_program_dir, exist_ok=True)
    out_file_path = out_program_dir / bc_path.name
    shutil.copyfile(bc_path, out_file_path)

In [9]:
def create_vector_embedding(bc_path, p_bar):
    out_path = bc_path.with_suffix('.vec')
    if out_path.exists():
        p_bar.update(1)
        return
    cmd = [
        '/mnt/c/Users/dogho/Desktop/Thesis/IRToVec/bin/ir2vec',
        '-sym',
        '-vocab',
        '/mnt/c/Users/dogho/Desktop/Thesis/IRToVec/IR2Vec/vocabulary/seedEmbeddingVocab-300-llvm10.txt',
        '-o',
        str(out_path.absolute()),
        '-level p',
        str(bc_path.absolute())
    ]
    resp = os.system(' '.join(cmd))
    p_bar.update(1)
    assert resp == 0

In [14]:
bc_program_paths = list(get_bc_program_paths(out_dir))
                       
with tqdm(total=len(bc_program_paths)) as p_bar:
    with ThreadPoolExecutor(max_workers=32) as pool:
        for bc_path in bc_program_paths:
            pool.submit(create_vector_embedding, bc_path, p_bar)

100%|██████████| 2560/2560 [00:00<00:00, 2619.41it/s]
